In [218]:
# Goal this excercise:  To read in a fixed width parse a fixed width file into seperate fields.  
# where each field is reformatted according to the key:value pair in a seperate file

# Step 1:  Read in the File and the Column Values

In [219]:
import pandas as pd
import re
from datetime import datetime as dt


#this will be our list of Field names and their widths
fields_df = pd.read_csv('Field_names.csv')
fields_df



# The dataframe displays column names in the column : Field Name.  
# the Column Size is just the width of each column that the fixed width file will be using for its columns
# clients often use large mainframe files that will use a key:value document like this to give 
# information on the fields in the fixed width file.  
    

,Field Name,Column Size
0,Filler,12
1,SSN,9
2,First Name,42
3,Last Name,23
4,Birth Date,25
5,Address Line 1,25
6,Address Line 2,22
7,City,11
8,State,17
9,Zip Code,11


In [220]:
# Next we read in the fixed width file
# read in the file as a pandas dataframe, using the pd.read_fwf()
# set the widths to be the fields_df Column Size field, this argument has to be a list, so we cast it as a list
# finally we set header=None in order not to mistake the first row for column names in the file
df =  pd.read_fwf('exampleHRData.csv', widths=list(fields_df['Column Size']), header=None) 

# here we are updating the dataframe's column names to match that of the key document
df.columns = fields_df['Field Name']

#since the first column is just a space filler, we drop this row
df.drop('Filler', axis = 1, inplace = True)
df




Field Name,SSN,First Name,Last Name,Birth Date,Address Line 1,Address Line 2,City,State,Zip Code,Employee ID
0,123456789,John,Doe123456789,4/23/1999,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,722192
1,822321115,John,Doe822321115,2/16/1999,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,840422
2,362208513,John,Doe362208513,12/23/1981,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,827434
3,505322754,John,Doe505322754,10/26/1990,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,304874
4,787795459,John,Doe787795459,4/25/1990,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,432056
...,...,...,...,...,...,...,...,...,...,...
494,123456789,John,Doe123456789,5/8/1993,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,346663
495,923723604,John,Doe923723604,6/24/1985,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,606121
496,776084656,John,Doe776084656,7/30/1989,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,854102
497,615567397,John,Doe615567397,5/20/1980,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,334743


# Step 2:  Inspect the newly parsed data

In [221]:
#Inspecting the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   SSN             499 non-null    int64 
 1   First Name      499 non-null    object
 2   Last Name       499 non-null    object
 3   Birth Date      499 non-null    object
 4   Address Line 1  499 non-null    object
 5   Address Line 2  499 non-null    object
 6   City            499 non-null    object
 7   State           499 non-null    object
 8   Zip Code        499 non-null    int64 
 9   Employee ID     499 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 39.1+ KB


In [222]:
# Looking at the info we can see a few updates are needed

#that the SSN's are not in the xxx-xx-xxxx format
# and also are in int64 datatype.  Meaning we could have the far left 0 for the ssn being cut short.  
# for example:   000123456 , might be accidently be 123456.   

# Also the Birth dates are not in the YYYY-MM-DD format, and they aren't date objects
# and with further inspection, we can see that there are some invalid values, like 2/30/1996

# Zip Codes are also int64 datatype.  We'll need an update similar to what we do to the SSN



# Step 3: Reformat columns that need updating

## Cleaning the SSN

In [223]:
#Cleaning up the SSNs we'll need to add a few 000's to the left of the SSN, then 
#take the right 9 digits to get the full number
#we can do this with a lambda function

df['SSN'] = df['SSN'].apply(lambda x: ("000000" + str(x))[-9:])
#now the SSN field is a string field, so we can use anohter lambda function and just 
#reformat it with a dash
df['SSN'] = df['SSN'].apply(lambda x: f"{x[:3]}-{x[3:5]}-{x[5:]}")
df['SSN']

0      123-45-6789
1      822-32-1115
2      362-20-8513
3      505-32-2754
4      787-79-5459
          ...     
494    123-45-6789
495    923-72-3604
496    776-08-4656
497    615-56-7397
498    162-98-4399
Name: SSN, Length: 499, dtype: object

In [224]:
## Cleaning the zip codes

df['Zip Code'] = df['Zip Code'].apply(lambda x: ("000000" +str(x))[-5:])
df['Zip Code']

0      77339
1      77339
2      77339
3      77339
4      77339
       ...  
494    77339
495    77339
496    77339
497    77339
498    77339
Name: Zip Code, Length: 499, dtype: object

## Cleaning the Birthdate

In [225]:
#we can make a function that will parse the birthdate and catch the bad february date in one sweep

def parse_dates(text):
    month = int(text.split('/')[0])
    day = int(text.split('/')[1])
    
    #we are just moving the date 2/30, to 2/28.  We could also just move it to march 1st. 
    if day > 28 and month == 2:
        day = 28
        
    year = int(text.split('/')[2])
    return dt(year, month, day)

#then just apply that function inside a lambda
df['Birth Date'] = df['Birth Date'].apply(lambda x: parse_dates(x))
df['Birth Date']

0     1999-04-23
1     1999-02-16
2     1981-12-23
3     1990-10-26
4     1990-04-25
         ...    
494   1993-05-08
495   1985-06-24
496   1989-07-30
497   1980-05-20
498   1986-03-22
Name: Birth Date, Length: 499, dtype: datetime64[ns]

# Review the updates

In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   SSN             499 non-null    object        
 1   First Name      499 non-null    object        
 2   Last Name       499 non-null    object        
 3   Birth Date      499 non-null    datetime64[ns]
 4   Address Line 1  499 non-null    object        
 5   Address Line 2  499 non-null    object        
 6   City            499 non-null    object        
 7   State           499 non-null    object        
 8   Zip Code        499 non-null    object        
 9   Employee ID     499 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 39.1+ KB


In [229]:
df

Field Name,SSN,First Name,Last Name,Birth Date,Address Line 1,Address Line 2,City,State,Zip Code,Employee ID
0,123-45-6789,John,Doe123456789,1999-04-23,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,722192
1,822-32-1115,John,Doe822321115,1999-02-16,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,840422
2,362-20-8513,John,Doe362208513,1981-12-23,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,827434
3,505-32-2754,John,Doe505322754,1990-10-26,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,304874
4,787-79-5459,John,Doe787795459,1990-04-25,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,432056
...,...,...,...,...,...,...,...,...,...,...
494,123-45-6789,John,Doe123456789,1993-05-08,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,346663
495,923-72-3604,John,Doe923723604,1985-06-24,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,606121
496,776-08-4656,John,Doe776084656,1989-07-30,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,854102
497,615-56-7397,John,Doe615567397,1980-05-20,123 Elvin Drive,Dwarf Street,Honolee,Texas,77339,334743


In [230]:
# we can see the fields are using the correct datatypes
# and looking at the output, they are correctly formatted.  
# so we can just save the file to a csv and call it a success.
df.to_csv('FormattedHR_File.csv')